# 🧠 Vector Quantized Variational Autoencoders con MedNIST Dataset

🎯 **¡Hoy serás tú quien entrene un modelo VQVAE!** (Vector Quantized Variational Autoencoder) para aprender representaciones discretas de imágenes médicas. El VQVAE es un modelo generativo súper poderoso que combina la codificación variacional con la cuantización vectorial, lo que permite aprender representaciones discretas de datos continuos. 💪

🏥 **¿Por qué es importante?** Este enfoque ha demostrado ser increíblemente eficaz en tareas de compresión y generación de imágenes médicas.

🚀 **¡Vas a ser capaz de entrenar un modelo VQVAE que puede reconstruir imágenes médicas como un profesional!** 

🎓 **Mi plan de entrenamiento para ti:** Entrenaremos nuestro modelo VQVAE para que sea capaz de reconstruir las imágenes de entrada. Trabajaremos con el conjunto de datos MedNIST disponible en MONAI (https://docs.monai.io/en/stable/apps.html#monai.apps.MedNISTDataset). 

⚡ **Para entrenar más rápido**, he seleccionado solo una de las clases disponibles ("HeadCT"), resultando en un conjunto de entrenamiento con 7999 imágenes 2D.

💡 **Dato curioso**: El VQVAE también se puede utilizar como un modelo generativo si entrenas un modelo autorregresivo (por ejemplo, PixelCNN, Transformer Decoder) en las representaciones latentes discretas del cuello de botella VQVAE. ¡Eso está fuera del alcance de este tutorial, pero es fascinante! 🌟


## 🔧 ¡Configurando tu entorno de trabajo!

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[tqdm]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

## 📦 ¡Importando nuestras herramientas mágicas!

In [ ]:
import os
import shutil
import tempfile
import time

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import torch
from torch.nn import L1Loss

from monai import transforms as mt
from monai.apps import MedNISTDataset
from monai.config import print_config
from monai.data import DataLoader, Dataset
from monai.utils import first, set_determinism, ensure_tuple
from monai.networks.nets import VQVAE

print_config()

## 📁 Estableciendo tu espacio de trabajo

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

## 🎲 ¡Configurando la reproducibilidad!

In [ ]:
set_determinism(42)

## 📥 ¡Descargando tus datos médicos!

In [ ]:
train_data = MedNISTDataset(root_dir=root_dir, section="training", download=True, seed=0)
train_datalist = [{"image": item["image"]} for item in train_data.data if item["class_name"] == "HeadCT"]
image_size = 64
batch_size = 16

train_transforms = mt.Compose(
    [
        mt.LoadImaged(keys=["image"]),
        mt.EnsureChannelFirstd(keys=["image"]),
        mt.ScaleIntensityRanged(keys=["image"], a_min=0.0, a_max=255.0, b_min=0.0, b_max=1.0, clip=True),
        mt.RandAffined(
            keys=["image"],
            rotate_range=[(-np.pi / 36, np.pi / 36), (-np.pi / 36, np.pi / 36)],
            translate_range=[(-1, 1), (-1, 1)],
            scale_range=[(-0.05, 0.05), (-0.05, 0.05)],
            spatial_size=[image_size, image_size],
            padding_mode="zeros",
            prob=0.5,
        ),
    ]
)
train_ds = Dataset(data=train_datalist, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4, persistent_workers=True)

## 👀 ¡Visualizando tus datos de entrenamiento!

In [ ]:
# Plot 3 examples from the training set
check_data = first(train_loader)
fig, ax = plt.subplots(nrows=1, ncols=3)
for image_n in range(3):
    ax[image_n].imshow(check_data["image"][image_n, 0, :, :], cmap="gray")
    ax[image_n].axis("off")

## 🔍 ¡Preparando tus datos de validación!

In [ ]:
val_data = MedNISTDataset(root_dir=root_dir, section="validation", download=True, seed=0)
val_datalist = [{"image": item["image"]} for item in val_data.data if item["class_name"] == "HeadCT"]
val_transforms = mt.Compose(
    [
        mt.LoadImaged(keys=["image"]),
        mt.EnsureChannelFirstd(keys=["image"]),
        mt.ScaleIntensityRanged(keys=["image"], a_min=0.0, a_max=255.0, b_min=0.0, b_max=1.0, clip=True),
    ]
)
val_ds = Dataset(data=val_datalist, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True, num_workers=4, persistent_workers=True)

## 🏗️ ¡Construyendo tu red VQVAE!

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")
model = VQVAE(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    channels=(256, 256),
    num_res_channels=256,
    num_res_layers=2,
    downsample_parameters=((2, 4, 1, 1), (2, 4, 1, 1)),
    upsample_parameters=((2, 4, 1, 1, 0), (2, 4, 1, 1, 0)),
    num_embeddings=256,
    embedding_dim=32,
).to(device)

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
l1_loss = L1Loss()

## 🚀 ¡Entrenando tu modelo VQVAE!

⏰ **¡Prepárate para la aventura!** Vamos a entrenar el modelo durante 100 épocas. 

⚡ **Tiempo estimado**: ~60 minutos (¡perfecto para tomar un café y ver cómo tu modelo aprende!) ☕

💡 **Mi consejo**: Mientras entrena, observa cómo evolucionan las métricas de pérdida. ¡Es fascinante ver el aprendizaje en tiempo real!

In [ ]:
max_epochs = 100
val_interval = 10
epoch_recon_loss_list = []
epoch_quant_loss_list = []
val_recon_epoch_loss_list = []
intermediary_images = []
n_example_images = 4

total_start = time.time()
for epoch in range(max_epochs):
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), ncols=110)
    progress_bar.set_description(f"Epoch {epoch}")
    for step, batch in progress_bar:
        images = batch["image"].to(device)
        optimizer.zero_grad(set_to_none=True)

        # model outputs reconstruction and the quantization error
        reconstruction, quantization_loss = model(images=images)

        recons_loss = l1_loss(reconstruction.float(), images.float())

        loss = recons_loss + quantization_loss

        loss.backward()
        optimizer.step()

        epoch_loss += recons_loss.item()

        progress_bar.set_postfix(
            {"recons_loss": epoch_loss / (step + 1), "quantization_loss": quantization_loss.item() / (step + 1)}
        )
    epoch_recon_loss_list.append(epoch_loss / (step + 1))
    epoch_quant_loss_list.append(quantization_loss.item() / (step + 1))

    if (epoch + 1) % val_interval == 0:
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_step, batch in enumerate(val_loader, start=1):
                images = batch["image"].to(device)

                reconstruction, quantization_loss = model(images=images)

                # get the first sample from the first validation batch for
                # visualizing how the training evolves
                if val_step == 1:
                    intermediary_images.append(reconstruction[:n_example_images, 0])

                recons_loss = l1_loss(reconstruction.float(), images.float())

                val_loss += recons_loss.item()

        val_loss /= val_step
        val_recon_epoch_loss_list.append(val_loss)

total_time = time.time() - total_start
print(f"train completed, total time: {total_time}.")

## 📈 ¡Analizando tus curvas de aprendizaje!

In [ ]:
plt.style.use("ggplot")
plt.title("Learning Curves", fontsize=20)
plt.plot(np.linspace(1, max_epochs, max_epochs), epoch_recon_loss_list, color="C0", linewidth=2.0, label="Train")
plt.plot(
    np.linspace(val_interval, max_epochs, int(max_epochs / val_interval)),
    val_recon_epoch_loss_list,
    color="C1",
    linewidth=2.0,
    label="Validation",
)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("Epochs", fontsize=16)
plt.ylabel("Loss", fontsize=16)
plt.legend(prop={"size": 14})
plt.show()

## 🎬 ¡Visualizando la evolución de tus imágenes generadas!

🌟 **¡Esta es mi parte favorita!** Aquí podrás ver cómo tu modelo mejora progresivamente a lo largo del entrenamiento. ¡Es como ver una película del aprendizaje de tu IA! 🎥

In [ ]:
# Plot every evaluation as a new line and example as columns
val_samples = np.linspace(val_interval, max_epochs, int(max_epochs / val_interval))
fig, ax = plt.subplots(nrows=len(val_samples), ncols=1, sharey=True)
ax = ensure_tuple(ax)
fig.set_size_inches(18.5, 30.5)
for image_n in range(len(val_samples)):
    reconstructions = torch.reshape(intermediary_images[image_n], (64 * n_example_images, 64)).T
    ax[image_n].imshow(reconstructions.cpu(), cmap="gray")
    ax[image_n].set_xticks([])
    ax[image_n].set_yticks([])
    ax[image_n].set_ylabel(f"Epoch {val_samples[image_n]:.0f}")

## 🖼️ ¡El momento de la verdad: tus imágenes reconstruidas!

🎉 **¡Llegó el momento más emocionante!** Vamos a comparar las imágenes originales con las que tu modelo ha reconstruido. ¿Qué tan bien crees que lo ha hecho? 🤔✨

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(images[0, 0].detach().cpu(), vmin=0, vmax=1, cmap="gray")
ax[0].axis("off")
ax[0].title.set_text("Inputted Image")
ax[1].imshow(reconstruction[0, 0].detach().cpu(), vmin=0, vmax=1, cmap="gray")
ax[1].axis("off")
ax[1].title.set_text("Reconstruction")
plt.show()

## 🧹 ¡Limpiando tu espacio de trabajo!

🗂️ **¡Hora de ser ordenado!** Eliminamos el directorio temporal si fue usado.

💡 **Mi recomendación**: Siempre es buena práctica limpiar los archivos temporales al finalizar tus experimentos. ¡Tu sistema te lo agradecerá! 🙏

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)